In [28]:
import pandas as pd

relevant_cols = ["primaryid", "caseid", "drug_seq", "role_cod", "drugname", "val_vbm", "route", "nda_num",
                 "dose_amt", "dose_unit", "dose_form", "prod_ai", "origin"]
drug_table: pd.DataFrame = pd.read_csv(filepath_or_buffer="../data/processed/drug_table.csv",
                                       nrows=10_000,
                                       usecols=relevant_cols)

drug_table.head()

,primaryid,caseid,drug_seq,role_cod,drugname,val_vbm,route,nda_num,dose_amt,dose_unit,dose_form,origin,prod_ai
0,34483284,3448328,1,PS,SUSTIVA,1,TRANSPLACENTAL,20972.0,NaN,NaN,NaN,drug12q4.txt,NaN
1,34483284,3448328,2,SS,NEVIRAPINE,1,TRANSPLACENTAL,NaN,NaN,NaN,NaN,drug12q4.txt,NaN
2,34483284,3448328,3,SS,VIRACEPT,1,TRANSPLACENTAL,NaN,NaN,NaN,NaN,drug12q4.txt,NaN
3,34483284,3448328,4,SS,COMBIVIR,1,TRANSPLACENTAL,NaN,NaN,NaN,NaN,drug12q4.txt,NaN
4,34483284,3448328,5,SS,RETROVIR,1,TRANSPLACENTAL,NaN,NaN,NaN,NaN,drug12q4.txt,NaN


In [29]:
for col in relevant_cols:
    print(f"column {col} has {drug_table[col].isna().sum()} null entries")

column primaryid has 0 null entries
column caseid has 0 null entries
column drug_seq has 0 null entries
column role_cod has 0 null entries
column drugname has 1 null entries
column val_vbm has 0 null entries
column route has 5006 null entries
column nda_num has 7510 null entries
column dose_amt has 5917 null entries
column dose_unit has 5916 null entries
column dose_form has 6949 null entries
column prod_ai has 10000 null entries
column origin has 0 null entries


dose_vbm does not seem to be an informative entry seeing that it's usually a null entry unlike val_vbm where every instance is recorded. For this reason we will drop it from the relevant column section to reduce noise.

In [37]:
# If an nda_number is present, we query that first against a dictionary that our pipeline will load on start-up.
nda_num = drug_table["nda_num"][0]
nda_num = int(nda_num)

In [39]:
# A helper function to count the number of null entries in a row
drug_table.isnull().sum(axis=1)

0       4
1       5
2       5
3       5
4       5
       ..
9995    3
9996    6
9997    6
9998    6
9999    6
Length: 10000, dtype: int64

In [41]:
test_series = drug_table.iloc[9995]

In [58]:
# Create QueryHolder class, which has the following fields:
# - query: str that gets passed to RxNav
# - nda_num: Optional[int] -> if it's present, query this against the NDA dict first
# - identifier: namedtuple of (primaryid, caseid, drug_seq). Will be used to map back to original data
# - role_code: Enum of all possible role codes, will be used for analysis later
# - backup_query: Optional[str] the same query with the product active ingredient replacing the drugname if no mapping was found... Only works if prod_ai present of course.


def create_query_holder(row: pd.Series) -> str: # TODO: Should return QueryHolder
    components = ["drugname", "route", "dose_amt", "dose_unit", "dose_form", "prod_ai"] # Note: very rarely drugname can be null!
    query = " ".join([str(row[c]) for c in components])
    query = query.replace("nan", "").rstrip()  # remove all nans from string and strip trailing whitespace.
    return query
    

In [59]:
create_query_holder(test_series)

'Allopurinol ORAL 100.0 MG'

In [62]:
legacy = pd.read_csv("../data/processed/legacy_set_fixed.csv", low_memory=False)

In [63]:
legacy.head()

,ISR,DRUG_SEQ,ROLE_COD,DRUGNAME,VAL_VBM,ROUTE,DOSE_VBM,DECHAL,RECHAL,LOT_NUM,EXP_DT,NDA_NUM,Unnamed: 12,origin
0,4204616,1004278786,PS,"MIFEPRISTONE TABLETS, 200 MG (DANCO LABS)",2,ORAL,"200 MG, ORAL",D,D,20002,20020331.0,20687.0,NaN,DRUG04Q1.TXT
1,4204616,1004279849,SS,MISOPROSTOL,1,VAGINAL,"800 MCG, VAGINAL",D,D,C2001748,NaN,NaN,NaN,DRUG04Q1.TXT
2,4214534,1004308602,PS,OXYCODONE HCL,1,NaN,NaN,D,D,NaN,NaN,20553.0,NaN,DRUG04Q1.TXT
3,4214534,1004349743,SS,"HYDROCODONE BITARTRATE (SIMILAR TO IND 59,175)...",2,NaN,NaN,D,D,NaN,NaN,NaN,NaN,DRUG04Q1.TXT
4,4214534,1004349748,C,ANALGESICS ( ),2,NaN,NaN,U,U,NaN,NaN,NaN,NaN,DRUG04Q1.TXT


In [68]:
legacy = legacy.drop(["Unnamed: 12"], axis=1)

In [71]:
legacy.to_csv("../data/processed/AERS_2004Q1_2012Q3.csv", index=False)

In [72]:
legacy.head()

,ISR,DRUG_SEQ,ROLE_COD,DRUGNAME,VAL_VBM,ROUTE,DOSE_VBM,DECHAL,RECHAL,LOT_NUM,EXP_DT,NDA_NUM,origin
0,4204616,1004278786,PS,"MIFEPRISTONE TABLETS, 200 MG (DANCO LABS)",2,ORAL,"200 MG, ORAL",D,D,20002,20020331.0,20687.0,DRUG04Q1.TXT
1,4204616,1004279849,SS,MISOPROSTOL,1,VAGINAL,"800 MCG, VAGINAL",D,D,C2001748,NaN,NaN,DRUG04Q1.TXT
2,4214534,1004308602,PS,OXYCODONE HCL,1,NaN,NaN,D,D,NaN,NaN,20553.0,DRUG04Q1.TXT
3,4214534,1004349743,SS,"HYDROCODONE BITARTRATE (SIMILAR TO IND 59,175)...",2,NaN,NaN,D,D,NaN,NaN,NaN,DRUG04Q1.TXT
4,4214534,1004349748,C,ANALGESICS ( ),2,NaN,NaN,U,U,NaN,NaN,NaN,DRUG04Q1.TXT
